## P2P Messaging
This notebook demonstrates the P2P messaging from 1 agent to another using Agent self topic in the RoutingRule.

Every Agent in the system is always listening to a special topic dedicated to it `agent_self:{agent_id}` defined in dsl.py under GuildTopics.

What we will demonstrate here -

**(default_topic) -> [Agent1/agent1:wire_message] -> (agent_inbox:agent2)**

**(agent_inbox:agent2) -> [Agent2/agent2:wire_message] -> (topic_2)**

In [1]:
# import AgentBuilder and GuildBuilder

from rustic_ai.core.agents.eip import BasicWiringAgent
from rustic_ai.core.guild.builders import AgentBuilder, GuildBuilder, RouteBuilder
from rustic_ai.core.guild.dsl import GuildTopics

# Create one agent that listens to the default topic and another that does not
agent1 = (
    AgentBuilder(BasicWiringAgent).set_name("Agent1").set_id("agent1").set_description("The first Agent").build_spec()
)
agent2 = (
    AgentBuilder(BasicWiringAgent)
    .set_name("Agent2")
    .set_id("agent2")
    .set_description("The second Agent")
    .listen_to_default_topic(False)
    .build_spec()
)

# Create the P2P route
p2p_route = (
    RouteBuilder(agent1)
    .set_destination_topics(GuildTopics.get_agent_inbox(agent2.id))  # Use the inbox topic for agent2
    .build()
)

# Create back to probe route
probe_route = RouteBuilder(agent2).set_destination_topics("topic_2").build()


# Create a guild with the two agents
guild = (
    GuildBuilder(
        "p2p_guild", "Point 2 Point Messaging", "A guild to demonstrate point-to-point messaging between agents."
    )
    .add_agent_spec(agent1)  # Add the first agent
    .add_agent_spec(agent2)  # Add the second agent
    .add_route(p2p_route)  # Add the P2P route
    .add_route(probe_route)  # Add the probe route
    .launch("myorg")
)

In [2]:
from rustic_ai.core.agents.testutils.probe_agent import ProbeAgent


probe_agent = (
    AgentBuilder(ProbeAgent)
    .set_name("ProbeAgent")
    .set_description("An agent to probe the guild")
    .add_additional_topic("topic_2")
    .build()
)

guild._add_local_agent(probe_agent)

In [3]:
from rustic_ai.core.guild.dsl import GuildTopics


probe_agent.publish_dict_with_guild_route(payload={"content": "Hello from ProbeAgent!"}, topic=GuildTopics.DEFAULT_TOPICS[0])

In [4]:
probe_agent.print_message_history(-1)

(default_topic) -> [Agent1/agent1:wire_message] -> (agent_inbox:agent2)
(agent_inbox:agent2) -> [Agent2/agent2:wire_message] -> (topic_2)
